In [1]:
# text = [str(k).lower() + " " + str(v).replace(')','').replace(';','').lower()  for k,v in json.items()]

In [2]:
from src.LocalDataManager.file_manager import FileManager

In [3]:
base_path= 'data/managed'
# base_path= 'D:/'

db_file='tinker.db'

metadata_dict={"name": "text",
      "ADSS": "text);",
      "version": 'number'}

In [4]:
manager = FileManager(base_path=base_path, 
                      db_file=db_file, 
                      verbose=False)

In [5]:
# Reading a file 
f = open(r'test/testfile.txt', 'rb')
# f = open(r'test.jpg', 'rb')

In [6]:
manager.hash_file(f)

'9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08'

In [7]:
manager.save_file(f, 'bronze', metadata=metadata_dict)

'testfile.txt already managed.'

In [8]:
df_files = manager.get_managed_files_df()

In [9]:
df_files

,hash,name,location,metadata_key,metadata_value
0,9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b82...,testfile.txt,data/managed\bronze,name,text
1,9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b82...,testfile.txt,data/managed\bronze,ADSS,text);
2,9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b82...,testfile.txt,data/managed\bronze,version,number


# head direct to the db class and play

In [10]:
from src.LocalDataManager.db.db_classes import File, Metadata, Base
from sqlalchemy.orm import Session

from sqlalchemy import create_engine

# Initialize the database engine with a connection string
engine = create_engine('sqlite:///:memory:')

# Create all tables in the database
Base.metadata.create_all(engine)

with Session(engine) as session:
    f1_hash = "hash1"
    f2_hash = "hash2"
    f3_hash = "hash3"
    file1 = File(
        hash=f1_hash,
        name="one",
        location="c:/test.txt",
        file_metadata=[Metadata(hash=f1_hash, metadata_key="test1_key", metadata_value="test1_value"),
                   Metadata(hash=f1_hash, metadata_key="test1_key2", metadata_value="test1_value2")],
    )
    file2 = File(
        hash=f2_hash,
        name="two",
        location="c:/test2.txt",
        file_metadata=[Metadata(hash=f2_hash, metadata_key="test2_key", metadata_value="test2_value"),
                   Metadata(hash=f2_hash, metadata_key="test2_key2", metadata_value="test2_value2")],
    )
    file3 = File(hash=f3_hash, name="three", location="c:/test3.txt")
    session.add_all([file1, file2, file3])
    session.commit()

In [11]:
from sqlalchemy import select

session = Session(engine)

stmt = select(File)

for file in session.scalars(stmt):
    print(file)

File(hash='hash1', name='one', location='c:/test.txt', metadata=[Metadata(hash='hash1', key='test1_key', value='test1_value'), Metadata(hash='hash1', key='test1_key2', value='test1_value2')])
File(hash='hash2', name='two', location='c:/test2.txt', metadata=[Metadata(hash='hash2', key='test2_key', value='test2_value'), Metadata(hash='hash2', key='test2_key2', value='test2_value2')])
File(hash='hash3', name='three', location='c:/test3.txt', metadata=[])


# close the connection to the db and delete the folders

In [12]:
import shutil
manager.close_connection()
conn.close()
shutil.rmtree(base_path)

AttributeError: 'FileManager' object has no attribute 'close_connection'

# misc

In [ ]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [ ]:
from sqlalchemy import create_engine
import os
engine = create_engine(os.path.join("sqlite:///",base_path, db_file), echo=True)
Base.metadata.create_all(engine)


2024-04-21 21:04:14,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-21 21:04:14,763 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("user_account")


2024-04-21 21:04:14,764 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE address (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES user_account (id)
)




2024-04-21 21:04:14,775 INFO sqlalchemy.engine.Engine [no key 0.00075s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00075s] ()


2024-04-21 21:04:14,778 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2024-04-21 21:04:15,630 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-21 21:04:15,632 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id


INFO:sqlalchemy.engine.Engine:INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id


2024-04-21 21:04:15,634 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')


...

INFO:sqlalchemy.engine.Engine:INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURNING id


2024-04-21 21:04:15,644 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('sandy@squirrelpower.org', 2)


INFO:sqlalchemy.engine.Engine:[insertmanyvalues 3/3 (ordered; batch not supported)] ('sandy@squirrelpower.org', 2)


2024-04-21 21:04:15,646 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User)

for user in session.scalars(stmt):
    print(user)

2024-04-21 21:04:22,571 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-21 21:04:22,573 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


INFO:sqlalchemy.engine.Engine:SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


2024-04-21 21:04:22,573 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00086s] ()


User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=3, name='patrick', fullname='Patrick Star')


In [ ]:
list(session.scalars(stmt))

2024-04-21 21:04:37,018 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


INFO:sqlalchemy.engine.Engine:SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


2024-04-21 21:04:37,018 INFO sqlalchemy.engine.Engine [cached since 14.45s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 14.45s ago] ()


[User(id=1, name='spongebob', fullname='Spongebob Squarepants'),
 User(id=2, name='sandy', fullname='Sandy Cheeks'),
 User(id=3, name='patrick', fullname='Patrick Star')]

AttributeError: type object 'User' has no attribute 'query'